In [1]:
import pandas as pd
from IPython.display import clear_output, Markdown, Math
import ipywidgets as widgets
import os
import unicodedata as uni
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet
import math
import pprint

pp=pprint.PrettyPrinter(indent=4)

# Naive Approach

* read in table

In [2]:
table = pd.read_csv('../Tools/emoji_descriptions.csv')
table.head()

,Unnamed: 0,code,character,description,description_de
0,0,126980,🀄,MAHJONG TILE RED DRAGON,MAHJONG FLIESE ROT DRACHE
1,1,129525,🧵,SPOOL OF THREAD,Spool Gewinde
2,2,129526,🧶,BALL OF YARN,BALL OF YARN
3,3,127183,🃏,PLAYING CARD BLACK JOKER,SPIELKARTE BLACK JOKER
4,4,129296,🤐,ZIPPER-MOUTH FACE,ZIPPER-MUND Gesicht


* todo: read in a lot of messages

In [3]:
messages = ["Hello, this is a testing message", "this is a very sunny day today, i am very happy"]

* using a Stemmer to get the main 'Part' of each word

In [4]:
ps = PorterStemmer()

In [5]:
stemmed_messages = []
for m in messages:
    words = word_tokenize(m)
    sm = []
    for w in words:
        sm.append(ps.stem(w))
    stemmed_messages.append(sm)

In [6]:
stemmed_messages

[['hello', ',', 'thi', 'is', 'a', 'test', 'messag'],
 ['thi',
  'is',
  'a',
  'veri',
  'sunni',
  'day',
  'today',
  ',',
  'i',
  'am',
  'veri',
  'happi']]

In [7]:
table.shape

(1027, 5)

* compare words to emoji descriptions

In [21]:
def evaluate_sentence(sentence, description_key = 'description', lang = 'eng'):
    
    tokenized_sentence = word_tokenize(sentence)
    n = len(tokenized_sentence)
    l = table.shape[0]
    matrix_list = []
    
    for index, row in table.iterrows():
        emoji_tokens = word_tokenize(row[description_key])
        m = len(emoji_tokens)

        mat = np.zeros(shape=(m,n))
        for i in range(len(emoji_tokens)):
            for j in range(len(tokenized_sentence)):
                syn1 = wordnet.synsets(emoji_tokens[i],lang=lang)
                if len(syn1) == 0:
                    continue
                w1 = syn1[0]
                #print(j, tokenized_sentence)
                syn2 = wordnet.synsets(tokenized_sentence[j], lang=lang)
                if len(syn2) == 0:
                    continue
                w2 = syn2[0]
                val = w1.wup_similarity(w2)
                if val is None:
                    continue
                mat[i,j] = val
        #print(row['character'], mat)
        matrix_list.append(mat)
            
    return matrix_list
    
    

* building a lookup table:

In [22]:
lookup = {}
emoji_set = []
for index, row in table.iterrows():
    lookup[index] = row['character']
    emoji_set.append(row['character'])

emoji_set = set(emoji_set)

* sorting

In [23]:
def predict(sentence, description_key='description', lang = 'eng', n=10, t=0.9):

    result = evaluate_sentence(sentence, description_key, lang)
    
    summed = np.argsort([-np.sum(x) for x in result])[0:n]
    max_val = np.argsort([-np.max(x) for x in result])[0:n]
    avg =  np.argsort([-np.mean(x) for x in result])[0:n]
    threshold = np.argsort([-len(np.where(x>t)[0]) / (x.shape[0] * x.shape[1]) for x in result])[0:n]
    
    # build a result table
    table_array = [[lookup[summed[i]], str(table.iloc[summed[i]][description_key]), 
                    lookup[max_val[i]], str(table.iloc[max_val[i]][description_key]),
                    lookup[avg[i]], str(table.iloc[avg[i]][description_key]),
                    lookup[threshold[i]], str(table.iloc[threshold[i]][description_key])] for i in range(n) ]
    
    
    table_frame = pd.DataFrame(table_array, columns=['summed', 'summed_description','max_val', 'max_val_description','avg', 'avg_description','threshold', 'threshold_description'])
    
    display(table_frame)


In [29]:
predict("I like to travel by train", description_key='description' , lang='eng')

,summed,summed_description,max_val,max_val_description,avg,avg_description,threshold,threshold_description
0,🤭,SMILING FACE WITH SMILING EYES AND HAND COVERI...,🤟,I LOVE YOU HAND SIGN,💐,BOUQUET,🚆,TRAIN
1,🔛,ON WITH EXCLAMATION MARK WITH LEFT RIGHT ARROW...,🇮,REGIONAL INDICATOR SYMBOL LETTER I,🍳,COOKING,🚸,CHILDREN CROSSING
2,🚮,PUT LITTER IN ITS PLACE SYMBOL,🚆,TRAIN,🚆,TRAIN,🚄,HIGH-SPEED TRAIN
3,🤪,GRINNING FACE WITH ONE LARGE AND ONE SMALL EYE,🚅,HIGH-SPEED TRAIN WITH BULLET NOSE,🎥,MOVIE CAMERA,🇮,REGIONAL INDICATOR SYMBOL LETTER I
4,🥰,SMILING FACE WITH SMILING EYES AND THREE HEARTS,🚄,HIGH-SPEED TRAIN,🎭,PERFORMING ARTS,🤟,I LOVE YOU HAND SIGN
5,🙌,PERSON RAISING BOTH HANDS IN CELEBRATION,🚸,CHILDREN CROSSING,🎹,MUSICAL KEYBOARD,🚅,HIGH-SPEED TRAIN WITH BULLET NOSE
6,🖖,RAISED HAND WITH PART BETWEEN MIDDLE AND RING ...,🛸,FLYING SAUCER,🧾,RECEIPT,🔹,SMALL BLUE DIAMOND
7,🔂,CLOCKWISE RIGHTWARDS AND LEFTWARDS OPEN CIRCLE...,🥾,HIKING BOOT,💏,KISS,📠,FAX MACHINE
8,😄,SMILING FACE WITH OPEN MOUTH AND SMILING EYES,🥏,FLYING DISC,🔥,FIRE,📡,SATELLITE ANTENNA
9,👉,WHITE RIGHT POINTING BACKHAND INDEX,🚏,BUS STOP,🎼,MUSICAL SCORE,📢,PUBLIC ADDRESS LOUDSPEAKER
